In [21]:
import datetime

In [29]:
datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

'2020-04-04_23-58-51'

In [1]:
from task1_train import *
from hparams import hparams, DEVICE_DICT, CITY_DICT, SCENE_DICT

In [2]:
print('{:^43s}'.format('==== Display hparams ===='))

display_dict(hparams)

dataset_name = 'TAU-urban-acoustic-scenes-2020-mobile-development'

dataset_base_path = os.path.join('datasets', dataset_name)

metadata_train_path = dataset_base_path + '/' + 'evaluation_setup' + '/' + 'fold1_train.csv'

# metadata_test_path = 'datasets/' + dataset_name + '/' + 'evaluation_setup' + '/' + 'fold1_test.csv' 

metadata_eval_path = dataset_base_path + '/' + 'evaluation_setup' + '/' + 'fold1_evaluate.csv' 

metadata_train = load_metadata(metadata_train_path, dataset_base_path)

# metadata_test = load_metadata(metadata_path=metadata_test_path)

metadata_eval = load_metadata(metadata_eval_path, dataset_base_path)

train_dataset_loader = DatasetLoader(metadata_train)
eval_dataset_loader = DatasetLoader(metadata_eval)

print(train_dataset_loader)

         ==== Display hparams ====         

                  fs | 44100               
           nsc_in_ms | 200                 
        mel_band_num | 80                  
           nov_in_ms | 100                 
                 nsc | 8820                
                 nov | 4410                
           max_epoch | 2                   
          batch_size | 16                  

[ Loaded datasets/TAU-urban-acoustic-scenes-2020-mobile-development/evaluation_setup/fold1_train.csv ] [ Length: 13965 ]
[ Loaded datasets/TAU-urban-acoustic-scenes-2020-mobile-development/evaluation_setup/fold1_evaluate.csv ] [ Length: 2970 ]
[ FileInfo(file_path='datasets/TAU-urban-acoustic-scenes-2020-mobile-development/audio/airport-lisbon-1000-40000-a.wav', city='lisbon', device='a', scene='airport'), ..., FileInfo(file_path='datasets/TAU-urban-acoustic-scenes-2020-mobile-development/audio/tram-lyon-1112-41171-s3.wav', city='lyon', device='s3', scene='tram') ]


In [3]:
random.shuffle(metadata_train)
metadata_train[:10]

[FileInfo(file_path='datasets/TAU-urban-acoustic-scenes-2020-mobile-development/audio/street_pedestrian-stockholm-157-4755-a.wav', city='stockholm', device='a', scene='street_pedestrian'),
 FileInfo(file_path='datasets/TAU-urban-acoustic-scenes-2020-mobile-development/audio/shopping_mall-stockholm-136-4116-a.wav', city='stockholm', device='a', scene='shopping_mall'),
 FileInfo(file_path='datasets/TAU-urban-acoustic-scenes-2020-mobile-development/audio/public_square-stockholm-120-3544-a.wav', city='stockholm', device='a', scene='public_square'),
 FileInfo(file_path='datasets/TAU-urban-acoustic-scenes-2020-mobile-development/audio/airport-barcelona-1-75-a.wav', city='barcelona', device='a', scene='airport'),
 FileInfo(file_path='datasets/TAU-urban-acoustic-scenes-2020-mobile-development/audio/street_pedestrian-london-149-4530-a.wav', city='london', device='a', scene='street_pedestrian'),
 FileInfo(file_path='datasets/TAU-urban-acoustic-scenes-2020-mobile-development/audio/bus-prague-1032

In [4]:
train_dataset_loader.start_loading()
for batch in train_dataset_loader.generator():
    mel_spectrogram_batch, file_path_list, city_label, device_label, scene_label = batch
    break

[ Start Loading ]


In [5]:
'''

inputs, labels = data

# 변화도(Gradient) 매개변수를 0으로 만들고
optimizer.zero_grad()

# 순전파 + 역전파 + 최적화를 한 후
outputs = net(inputs)
loss = criterion(outputs, labels)
loss.backward()
optimizer.step()

# 통계를 출력합니다.
running_loss += loss.item()
if i % 2000 == 1999:    # print every 2000 mini-batches
    print('[%d, %5d] loss: %.3f' %
          (epoch + 1, i + 1, running_loss / 2000))
    running_loss = 0.0
    
'''

"\n\ninputs, labels = data\n\n# 변화도(Gradient) 매개변수를 0으로 만들고\noptimizer.zero_grad()\n\n# 순전파 + 역전파 + 최적화를 한 후\noutputs = net(inputs)\nloss = criterion(outputs, labels)\nloss.backward()\noptimizer.step()\n\n# 통계를 출력합니다.\nrunning_loss += loss.item()\nif i % 2000 == 1999:    # print every 2000 mini-batches\n    print('[%d, %5d] loss: %.3f' %\n          (epoch + 1, i + 1, running_loss / 2000))\n    running_loss = 0.0\n    \n"

In [19]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import torch

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

class CRNN_Net(nn.Module):
    def __init__(self):
        super(CRNN_Net, self).__init__()
        self.conv = nn.Conv2d(1, 256, kernel_size=(80, 11), stride=5)
        self.rnn_1 = nn.RNN(256, 256)
        self.rnn_2 = nn.RNN(256, 128)
#         self.rnn_3 = nn.RNN(128, 64)
#         self.rnn_4 = nn.RNN(64, 32)
        self.fc = nn.Linear(128 * 19, len(SCENE_DICT))
        
        # (B, M[80], H[1], T[101]) => (B, M[80], H[256], T[21])
        

    def forward(self, tensor):
        tensor = torch.Tensor(tensor).unsqueeze(1)
        tensor = F.relu(self.conv(tensor))
        tensor = tensor.squeeze(2)
        tensor = tensor.permute(2, 0, 1) # (B, H, T) => (T, B, H)
        tensor, _ = self.rnn_1(tensor)
        tensor, _ = self.rnn_2(tensor)
#         tensor, _ = self.rnn_3(tensor)
#         tensor, _ = self.rnn_4(tensor)
        tensor = tensor.permute(1, 2, 0) # (T, B, H) => (B, H, T)
#         tensor = tensor.permute()
        tensor = tensor.reshape(tensor.shape[0], -1)

        tensor = self.fc(tensor)
        tensor = F.log_softmax(tensor, dim=-1)

        return tensor

net = CRNN_Net()

In [20]:
net(batch[0]).shape

torch.Size([16, 10])